In [1]:
''' 
AREA_output_statistics.ipynb 
By Natalie Bratset

This notebook reads the AREA output file and generates a bunch of statistics based on that output. 
Make sure to change the export_folder variable to your exports folder!

I'm planning to convert this to a .py file after writing/testing in this notebook.
'''

import os
import pandas as pd
import numpy as np
import matplotlib.patches as mplpatches
import matplotlib.pyplot as plt
# from scipy import stats
# from scipy.interpolate import splrep, splev
# import plotly.express as px

area_results_path = os.path.expanduser("/scratch/Shares/dowell/temp/ChrisO/PSEA/AREA_fast/output/AREA_output/all_chrom_comorbidthreshold0/area_scores_20250808-192325.adjpval.csv")  # Update with AREA rsults we want to look at
gene_ref_path = os.path.expanduser("/scratch/Shares/dowell/temp/ChrisO/PSEA/AREA/testdata/genes.csv")
export_folder = "/scratch/Users/nabr7467/AREA_LE/leadingedge_spline/" # Change this to your export folder

area_results = pd.read_csv(area_results_path)
gene_ref = pd.read_csv(gene_ref_path)


In [ ]:
################################## USER INPUTS HERE ###########################################

# this is for setting p_val cutoffs and whatnot
# P value Columns: pval,p_value_bonf,p_value_holm,p_value_BenjaminiHochberg,p_value_BenjaminiYekutieli

##############################################################################################

In [35]:
def minus_log_pval(df, col):
    ''' What this funct do'''
    #checks for a 0 pvalue to handle
    if 0 in area_results[col]:
        min_nonzero_pval = np.min(area_results[col][area_results[col] != 0]) # finds the minimum pvalue that is not 0
        adjusted_pval = min_nonzero_pval/(10**20)                            # this value is the adjusted pvalue for AREA results with a pvalue of 0. This is such that we can take the -log10(pval)
        df.loc[df[col] == 0, col] = adjusted_pval                            # change 0s to a smaller val than minimum
    
    df['minuslog10pvalue'] = -np.log10(df[col])
    
    return df

def sum_comorbids():
    pass

def manhattan_plot(df):
    '''Input: pandas grouped dataframe by binary_atribute with a pre-calculated minuslog10pvalue col.'''
    pass

## MAIN ##
df = minus_log_pval(area_results,'p_value_bonf')
grouped = df.groupby(('binary_attribute'))

# for num, (name, group) in enumerate(grouped):
#     print(num)


In [36]:
# #for -log10(x)

# for i in np.arange(1,1000):
#     print(i,10.0**(-i))
#     print(np.log10(10.0**(-i)))

# # max possible x is 10^-323

pval = 1.0090442940654658e-78
min_val = pval/(10**20)
# min_val = 10.0**(-323)

print(np.log10(pval),np.log10(min_val))

# for i in np.arange(1,20):
#     print(i, np.log(10**i))
      

-77.99608976909907 -97.99608976909907
